## Bounding Boxes Visualization
Option: enable Non-maximum suppression to decrease bounding box overlap

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time
import collections
import json
import scipy.misc

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline

sys.path.insert(0,os.path.abspath(".."))
from utils import label_map_util
from utils import visualization_utils as vis_util

## Helper function to load image

In [2]:
def load_image_into_numpy_array(image):
    im_width, im_height = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

## Define detection thresholds

In [3]:
breading_index ={1: {'id': 1, 'name': 'bin',         'threshold':0.5},
                 2: {'id': 2, 'name': 'bowl',        'threshold':0.5},
                 3: {'id': 3, 'name': 'bucket',      'threshold':0.5},
                 4: {'id': 4, 'name': 'car',         'threshold':0.5},
                 5: {'id': 5, 'name': 'cup',         'threshold':0.5},
                 6: {'id': 6, 'name': 'jar',         'threshold':0.5},
                 7: {'id': 7, 'name': 'pottedplant', 'threshold':0.5},
                 8: {'id': 8, 'name': 'tire',        'threshold':0.5},
                 9: {'id': 9, 'name': 'vase',        'threshold':0.5}}

## Get the list of directories or subdistricts
That contains path_to_directory, province, district, and subdistrict 

In [4]:
directories = []

province = 'กรุงเทพมหานคร'
DIR = '../data/GSV'

districts = os.listdir(os.path.join(DIR, province))
for district in districts:
    subdists = os.listdir(os.path.join(DIR, province, district))
    for subdist in subdists:
        directories.append([DIR, province, district, subdist])

print(directories[0])

['../data/GSV', 'กรุงเทพมหานคร', 'เขตดุสิต', 'แขวงสวนจิตรลดา']


In [5]:
# sess = tf.InteractiveSession()

last = 0
for k, directory in enumerate(directories[last:]):
    DIR, province, district, subdist = directory
    directory = os.path.join(DIR, province, district, subdist)
    
    image_path = os.path.join(directory, 'original')
    if not os.path.exists(image_path): continue
    
    FILES = os.listdir(image_path)
    
    print('Index:',k+last,':\tDirectory:',directory, '\tTotal:',len(FILES),'images')
    
    if not os.path.exists(os.path.join(directory, 'bnd_img')):
        os.makedirs(os.path.join(directory, 'bnd_img'))        
    
    count_bnd_img = 0
    json_list = os.listdir(os.path.join(directory, 'json'))
    for json_file in tqdm_notebook(json_list):
        PATH = os.path.join(directory, 'json', json_file)
        with open(PATH) as file:
            data = json.load(file)

        if(len(data['breading_boxes']) == 0): continue

        boxes, classes, scores = [], [], []
        for index in data['breading_boxes']:
            boxes.append(data['boxes'][index]['bndbox'])
            scores.append(data['boxes'][index]['score'])
            classes.append(data['boxes'][index]['cls_id'])

        boxes = np.array(boxes)
        scores = np.array(scores)
        classes = np.array(classes)
        
        image = Image.open(os.path.join(directory, 'original', data['image_name']))
        image_np = load_image_into_numpy_array(image)
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            boxes,
            classes,
            scores,
            breading_index,
            use_normalized_coordinates=True,
            line_thickness=1,
            max_boxes_to_draw=len(scores),
            min_score_thresh=0)

        scipy.misc.imsave(os.path.join(directory, 'bnd_img', data['image_name'][:-4]+'_bnd.jpg'), image_np)
        count_bnd_img +=1
        
    print('Bnd:', count_bnd_img, 'images')

# sess.close()    

Index: 0 :	Directory: ../data/GSV/กรุงเทพมหานคร/เขตดุสิต/แขวงสวนจิตรลดา 	Total: 1 images


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))


Bnd: 1 images


/home/poom/.local/lib/python3.5/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


## Option: Non-maximum suppression

In [6]:
#     nms = tf.image.non_max_suppression(boxes, scores, len(scores), iou_threshold=0.3)
#     nms = sess.run(nms).astype(int)

#     cls_count = {}
#     for cls in classes[nms]:
#         name = breading_index[cls]['name']
#         if(name not in cls_count):
#             cls_count[name] = 0
#         cls_count[name] += 1

#     data['nms_indexes'] = nms.tolist()
#     data['nms_count'] = cls_count

#     with open(PATH, 'w') as file:
#         file.write(json.dumps(data, sort_keys=True, indent=4, separators=(',', ': ')))
        
    # ======================= Visualize bounding box =====================================

#     image = Image.open(os.path.join(directory, 'original', data['image_name']))
#     image_np = load_image_into_numpy_array(image)
#     vis_util.visualize_boxes_and_labels_on_image_array(
#         image_np,
#         boxes[nms],
#         classes[nms],
#         scores[nms],
#         breading_index,
#         use_normalized_coordinates=True,
#         line_thickness=1,
#         max_boxes_to_draw=len(scores),
#         min_score_thresh=0)

#     scipy.misc.imsave(os.path.join(directory, 'bnd_img', data['image_name'][:-4]+'_nms_no_label.jpg'), image_np)
